In [31]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torchvision
from datasets import load_dataset
import matplotlib.pyplot as plt

In [60]:
# prepare dataset
class CatsDataset(Dataset):
    def __init__(self, train=True):
        if train:
            temp_dataset = load_dataset("cifar10")['train']
        else:
            temp_dataset = load_dataset("cifar10")['test']
        
        main_dataset = []
        cats = []
        for i in range(len(temp_dataset)):
            # store cats
            if temp_dataset[i]['label'] == 3:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']).expand((3, -1, -1)) / 255.0
                cats.append([img, torch.tensor([0])])

            # add other images to dataset
            else:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']).expand((3, -1, -1)) / 255.0
                main_dataset.append([img, torch.tensor([1])])

        # Repeat dataset2 nine times
        repeated_datasets = [cats] * 9

        # Concatenate dataset1 with repeated dataset2
        self.dataset = ConcatDataset([main_dataset] + repeated_datasets)

    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        image = torchvision.transforms.RandomHorizontalFlip()(image)
        image = torchvision.transforms.RandomResizedCrop((32, 32), scale=(4/5, 5/4), ratio=(4/5, 5/4))(image)
        return (image, label)

In [61]:
train_dataset = CatsDataset(train=True)
len(train_dataset)

90000

In [62]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=256,
    shuffle=True
)

In [98]:
dev = torch.device("cpu")
if torch.cuda.is_available():
    dev = torch.device("cuda")
elif torch.backends.mps.is_available():
    dev = torch.device("mps")

print(f"Using device: {dev}")

Using device: mps


In [99]:
activation = nn.Tanh()

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(3072, 2048),
    activation,
    nn.Linear(2048, 2048),
    activation,
    nn.Linear(2048, 256),
    activation,
    nn.Linear(256, 2)
).to(dev)

In [102]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [103]:
for epoch in range(32):
    s = 0
    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(dev)
        labels = labels.to(dev)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        labels = labels.view(-1)
        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        s += loss.item()
    
    print(f"Epoch {epoch+1} Loss: {s/len(train_dataloader)}")

torch.save(model.to(dev).state_dict(), "model.pt")

Epoch 1 Loss: 0.5775416967543688
Epoch 2 Loss: 0.5220481100576845
Epoch 3 Loss: 0.5013280079107393
Epoch 4 Loss: 0.4826268781484528
Epoch 5 Loss: 0.46739222143184056
Epoch 6 Loss: 0.4581975602460178
Epoch 7 Loss: 0.44456527077338914
Epoch 8 Loss: 0.43892995539036667
Epoch 9 Loss: 0.4281998019326817
Epoch 10 Loss: 0.4216685225679116
Epoch 11 Loss: 0.4160304289649833
Epoch 12 Loss: 0.41350307188589464
Epoch 13 Loss: 0.40522246003489604
Epoch 14 Loss: 0.3994879905473102
Epoch 15 Loss: 0.39519649439237337
Epoch 16 Loss: 0.38709938136691396
Epoch 17 Loss: 0.3831336766650731
Epoch 18 Loss: 0.38111260397867724
Epoch 19 Loss: 0.3737461343407631
Epoch 20 Loss: 0.3676795715635473
Epoch 21 Loss: 0.3681715832684528
Epoch 22 Loss: 0.3637932574545795
Epoch 23 Loss: 0.3602344878864559
Epoch 24 Loss: 0.3558131206611341
Epoch 25 Loss: 0.35079673085023055
Epoch 26 Loss: 0.3479595072567463
Epoch 27 Loss: 0.3447911680083383
Epoch 28 Loss: 0.3400278375907378
Epoch 29 Loss: 0.33455023779110477
Epoch 30 Loss

In [104]:
test_dataset = CatsDataset(train=False)

In [105]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=256,
    shuffle=True
)

In [106]:
good = 0
a = 0
for i, data in enumerate(test_dataloader):
    # Every data instance is an input + label pair
    inputs, labels = data
    inputs = inputs.to(dev)
    labels = labels.to(dev)
    outputs = model(inputs)
    labels = labels.view(-1)
    
    for i in range(len(outputs)):
        good += torch.argmax(outputs[i]) == labels[i]
        a += 1

print(good / a)

tensor(0.8142, device='mps:0')
